# ==================================================================================
# 1. Python code to dynamically create the Instagram script 
# ==================================================================================

In [ ]:
import pandas as pd
import json
import re
import inspect

# Load schema       - Alt Path (/mnt/c/Users/arodilla/Downloads/Merged_structures_IG.csv)
schema_df = pd.read_csv('/mnt/c/Users/arodilla/OneDrive - Universitat de Barcelona/BSC/WHAT-IF/SCHEMA_DATA/Merged_structures_IG.csv')
schema_df.columns = schema_df.columns.str.strip()
schema_df = schema_df.dropna(subset=["variable", "value"])

# Helper function to flatten the schema and map to simplified column names
def flatten_schema(schema: dict, parent_key: str = '') -> dict:
    """Recursively flattens nested schema dicts to dot notation paths and returns user-friendly column names."""
    items = {}
    for k, v in schema.items():
        new_key = f"{parent_key}.{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_schema(v, new_key))
        else:
            items[new_key] = v
    return items

def generate_df_function_by_json(json_name: str, group: pd.DataFrame) -> str:
    """
    Generates a parsing function per json_name, using 'row_path' for row selection
    and up to 5 levels of nested 'col_path_*' fields for column mapping.
    """
    json_name_no_ext = json_name.replace(".json", "")
    row_path = group["row_path"].iloc[0]  # updated to use new column

    # Clean function name
    func_name = re.sub(r"\W|^(?=\d)", "_", json_name_no_ext.lower()) + "_df"

    col_paths = {}
    col_path_fields = ["col_path_1", "col_path_2", "col_path_3", "col_path_4", "col_path_5"]

    for _, row in group.iterrows():
        # Extract all non-null parts of the path
        path_parts = [str(row[col]) for col in col_path_fields if pd.notna(row.get(col))]
        if not path_parts:
            continue

        # The column name is the last part
        col_name = path_parts[-1]
        col_name = re.sub(r"\W|^(?=\d)", "_", col_name.strip().replace(" ", "_"))

        json_path = ".".join(path_parts)
        col_paths[col_name] = [json_path]

    # Start function definition
    lines = []
    lines.append(f"def {func_name}(file_input: list[str]) -> pd.DataFrame:")
    lines.append(f'    data = read_json(file_input, ["*/{json_name_no_ext}.json"])')
    lines.append("")
    lines.append("    df = parse_json(data,")
    lines.append(f'        row_path=["$.{row_path}"],')
    lines.append("        col_paths=dict(")

    for col_name, path in col_paths.items():
        lines.append(f'        {col_name} = {path},')

    lines.append("        )")
    lines.append("    )")
    lines.append("")

    lines.append('    if "time" in df.columns:')
    lines.append('        df["date"] = pd.to_datetime(df["time"], unit="s").dt.strftime("%Y-%m-%d %H:%M:%S")')
    lines.append('        df = df.sort_values("date")')
    lines.append("")

    lines.append("    return df\n")

    return "\n".join(lines)

def generate_donation_flow_function_explicit(schema_df: pd.DataFrame) -> str:
    """
    Generates an explicit version of create_donation_flow that includes hardcoded try/except
    blocks per known extractor function.
    """
    lines = []
    lines.append("def create_donation_flow(file_input: list[str]):")
    lines.append('    """')
    lines.append("    Creates a donation flow for Instagram data, explicitly trying each extractor function.")
    lines.append("    Only creates tables for data that's available in the provided files.")
    lines.append('    """')
    lines.append("    tables = []")
    lines.append("    #print(file_input)\n")

    grouped = schema_df.groupby("json_name")

    for json_name, group in grouped:
        #name_base = json_name.replace(".json", "").replace("'","_").replace("-","_").lower()
        name_base = re.sub(r"\W|^(?=\d)", "_", json_name.replace(".json", "").lower())
        func_name = re.sub(r"\W|^(?=\d)", "_", name_base) + "_df"
        table_name = name_base

        # Basic fallback titles (could be improved by a proper map)
        #raw_title = group["value"].iloc[0] if pd.notna(group["value"].iloc[0]) else table_name.replace("_", " ").title()
        #escaped_title = raw_title.replace('"', '\\"').replace("'", "\\'")
        escaped_title = table_name
        english_title = escaped_title
        dutch_title = escaped_title  # or translate if needed

        lines.append(f"    # {english_title}")
        lines.append("    try:")
        lines.append(f"        {table_name}_table = donation_table(")
        lines.append(f'            name="{table_name}",')
        lines.append(f"            df={func_name}(file_input),")
        lines.append(f'            title={{\"en\": \"{english_title}\", \"nl\": \"{dutch_title}\"}},')
        lines.append("        )")
        lines.append(f"        tables.append({table_name}_table)")
        lines.append("    except Exception as e:")
        lines.append(f'        #print(f\"Skipping {table_name}: {{e}}\")')
        lines.append("        pass\n")

    lines.append("    # Only create the donation flow if we have at least one table")
    lines.append("    if tables:")
    lines.append("        return donation_flow(")
    lines.append('            id=\"instagram\",')
    lines.append("            tables=tables")
    lines.append("        )")
    lines.append("    else:")
    lines.append('        #print(\"No tables could be generated from the provided files\")')
    lines.append("        return None")

    return "\n".join(lines)

# Group by json_name (1 table per JSON file)
generated_functions = [
    generate_df_function_by_json(json_name, group)
    for json_name, group in schema_df.groupby("json_name")
]

# Function to create donation flow (remains the same)
def create_donation_flow(file_input: list[str]):
    """
    Creates a donation flow using pre-defined extractors applied to file_input ZIP.
    """
    tables = []

    # List of all available extractor functions dynamically
    extraction_functions = {}

    for row in schema_df.iterrows():
        filename = row[1]["variable"].split("/")[-1]
        func_name = re.sub(r"\W|^(?=\d)", "_", filename.replace(".json", "").lower()) + "_df"
        extraction_functions[filename] = globals().get(func_name)

    # Run the extractors
    for filename, extractor_func in extraction_functions.items():
        try:
            df = pd.DataFrame(extractor_func(file_input))
            if not df.empty:
                table_name = filename.replace(".json", "").capitalize()
                tables.append(
                    donation_table(
                        name=table_name,
                        df=df,
                        title={"en": table_name}
                    )
                )
        except Exception as e:
            logger.error(f"Error running {extractor_func.__name__}: {e}")

    return donation_flow(
        id="Instagram",
        tables=tables
    )

# Save the generated functions to a file
all_code = "\n\n".join(generated_functions)
# Get the source code of the create_donation_flow function as a string
donation_flow_function_str = generate_donation_flow_function_explicit(schema_df)

# Save the generated functions to a file
all_code = "\n\n".join(generated_functions)
with open("src/framework/processing/py/port/helpers/instagram_generated_extractors.py", "w", encoding="utf-8") as f:
    f.write("# Auto-generated Instagram extractors\n\n")
    f.write("import pandas as pd\n")
    f.write("import logging\n")
    f.write("from port.helpers.donation_flow import donation_table, donation_flow\n")
    f.write("from port.helpers.readers import read_json\n")
    f.write("from port.helpers.parsers import parse_json\n\n")
    f.write("logger = logging.getLogger(__name__)\n\n")
    f.write(all_code)
    f.write("\n\n")
    f.write(donation_flow_function_str)

# ==================================================================================
# 2. Python code to dynamically create the Facebook script 
# ==================================================================================

In [ ]:
import pandas as pd
import json
import re
import inspect
import keyword 

# Load schema       - Alt Path (/mnt/c/Users/arodilla/Downloads/Merged_structures_IG.csv)
schema_df = pd.read_csv('/mnt/c/Users/arodilla/OneDrive - Universitat de Barcelona/BSC/WHAT-IF/SCHEMA_DATA/Merged_structures_FB.csv')
schema_df.columns = schema_df.columns.str.strip()
schema_df = schema_df.dropna(subset=["variable", "value"])

# Helper function to flatten the schema and map to simplified column names
def flatten_schema(schema: dict, parent_key: str = '') -> dict:
    """Recursively flattens nested schema dicts to dot notation paths and returns user-friendly column names."""
    items = {}
    for k, v in schema.items():
        new_key = f"{parent_key}.{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_schema(v, new_key))
        else:
            items[new_key] = v
    return items

def generate_df_function_by_json(json_name: str, group: pd.DataFrame) -> str:
    """
    Generates a parsing function per json_name, using 'row_path' for row selection
    and up to 5 levels of nested 'col_path_*' fields for column mapping.
    """
    json_name_no_ext = json_name.replace(".json", "")
    row_path = group["row_path"].iloc[0]  # updated to use new column

    # Clean function name
    func_name = re.sub(r"\W|^(?=\d)", "_", json_name_no_ext.lower()) + "_df"

    col_paths = {}
    col_path_fields = ["col_path_1", "col_path_2", "col_path_3", "col_path_4", "col_path_5"]

    for _, row in group.iterrows():
        # Extract all non-null parts of the path
        path_parts = [str(row[col]) for col in col_path_fields if pd.notna(row.get(col))]
        if not path_parts:
            continue

        # The column name is the last part
        col_name = path_parts[-1]
        col_name = re.sub(r"\W|^(?=\d)", "_", col_name.strip().replace(" ", "_"))

        # If it's a Python keyword, add underscore
        if keyword.iskeyword(col_name):
            col_name += "_"

        json_path = ".".join(path_parts)
        col_paths[col_name] = [json_path]

    # Start function definition
    lines = []
    lines.append(f"def {func_name}(file_input: list[str]) -> pd.DataFrame:")
    lines.append(f'    data = read_json(file_input, ["*/{json_name_no_ext}.json"])')
    lines.append("")
    lines.append("    df = parse_json(data,")
    lines.append(f'        row_path=["$.{row_path}"],')
    lines.append("        col_paths=dict(")

    for col_name, path in col_paths.items():
        lines.append(f'        {col_name} = {path},')

    lines.append("        )")
    lines.append("    )")
    lines.append("")

    lines.append('    if "time" in df.columns:')
    lines.append('        df["date"] = pd.to_datetime(df["time"], unit="s").dt.strftime("%Y-%m-%d %H:%M:%S")')
    lines.append('        df = df.sort_values("date")')
    lines.append("")

    lines.append("    return df\n")

    return "\n".join(lines)

def generate_donation_flow_function_explicit(schema_df: pd.DataFrame) -> str:
    """
    Generates an explicit version of create_donation_flow that includes hardcoded try/except
    blocks per known extractor function.
    """
    lines = []
    lines.append("def create_donation_flow(file_input: list[str]):")
    lines.append('    """')
    lines.append("    Creates a donation flow for Facebook data, explicitly trying each extractor function.")
    lines.append("    Only creates tables for data that's available in the provided files.")
    lines.append('    """')
    lines.append("    tables = []")
    lines.append("    #print(file_input)\n")

    grouped = schema_df.groupby("json_name")

    for json_name, group in grouped:
        #name_base = json_name.replace(".json", "").replace("'","_").replace("-","_").lower()
        name_base = re.sub(r"\W|^(?=\d)", "_", json_name.replace(".json", "").lower())
        func_name = re.sub(r"\W|^(?=\d)", "_", name_base) + "_df"
        table_name = name_base

        # Basic fallback titles (could be improved by a proper map)
        #raw_title = group["value"].iloc[0] if pd.notna(group["value"].iloc[0]) else table_name.replace("_", " ").title()
        #escaped_title = raw_title.replace('"', '\\"').replace("'", "\\'")
        escaped_title = table_name
        english_title = escaped_title
        dutch_title = escaped_title  # or translate if needed

        lines.append(f"    # {english_title}")
        lines.append("    try:")
        lines.append(f"        {table_name}_table = donation_table(")
        lines.append(f'            name="{table_name}",')
        lines.append(f"            df={func_name}(file_input),")
        lines.append(f'            title={{\"en\": \"{english_title}\", \"nl\": \"{dutch_title}\"}},')
        lines.append("        )")
        lines.append(f"        tables.append({table_name}_table)")
        lines.append("    except Exception as e:")
        lines.append(f'        #print(f\"Skipping {table_name}: {{e}}\")')
        lines.append("        pass\n")

    lines.append("    # Only create the donation flow if we have at least one table")
    lines.append("    if tables:")
    lines.append("        return donation_flow(")
    lines.append('            id=\"facebook\",')
    lines.append("            tables=tables")
    lines.append("        )")
    lines.append("    else:")
    lines.append('        #print(\"No tables could be generated from the provided files\")')
    lines.append("        return None")

    return "\n".join(lines)

# Group by json_name (1 table per JSON file)
generated_functions = [
    generate_df_function_by_json(json_name, group)
    for json_name, group in schema_df.groupby("json_name")
]

# Function to create donation flow (remains the same)
def create_donation_flow(file_input: list[str]):
    """
    Creates a donation flow using pre-defined extractors applied to file_input ZIP.
    """
    tables = []

    # List of all available extractor functions dynamically
    extraction_functions = {}

    for row in schema_df.iterrows():
        filename = row[1]["variable"].split("/")[-1]
        func_name = re.sub(r"\W|^(?=\d)", "_", filename.replace(".json", "").lower()) + "_df"
        extraction_functions[filename] = globals().get(func_name)

    # Run the extractors
    for filename, extractor_func in extraction_functions.items():
        try:
            df = pd.DataFrame(extractor_func(file_input))
            if not df.empty:
                table_name = filename.replace(".json", "").capitalize()
                tables.append(
                    donation_table(
                        name=table_name,
                        df=df,
                        title={"en": table_name}
                    )
                )
        except Exception as e:
            logger.error(f"Error running {extractor_func.__name__}: {e}")

    return donation_flow(
        id="Facebook",
        tables=tables
    )

# Save the generated functions to a file
all_code = "\n\n".join(generated_functions)
# Get the source code of the create_donation_flow function as a string
donation_flow_function_str = generate_donation_flow_function_explicit(schema_df)

# Save the generated functions to a file
all_code = "\n\n".join(generated_functions)
with open("src/framework/processing/py/port/helpers/facebook_generated_extractors.py", "w", encoding="utf-8") as f:
    f.write("# Auto-generated Facebook extractors\n\n")
    f.write("import pandas as pd\n")
    f.write("import logging\n")
    f.write("from port.helpers.donation_flow import donation_table, donation_flow\n")
    f.write("from port.helpers.readers import read_json\n")
    f.write("from port.helpers.parsers import parse_json\n\n")
    f.write("logger = logging.getLogger(__name__)\n\n")
    f.write(all_code)
    f.write("\n\n")
    f.write(donation_flow_function_str)

# ==================================================================================
# 3. Python code to dynamically create the TikTok script 
# ==================================================================================

In [ ]:
import pandas as pd
import json
import inspect
from typing import List
from pathlib import Path

# -----------------------
# Utility Functions
# -----------------------
def get_in(d: dict, *keys):
    for key in keys:
        if isinstance(d, dict):
            d = d.get(key)
        else:
            return None
    return d

def get_list(d: dict, *keys):
    val = get_in(d, *keys)
    return val if isinstance(val, list) else []

def get_dict(d: dict, *keys):
    val = get_in(d, *keys)
    return val if isinstance(val, dict) else {}

def snake_case(name: str) -> str:
    return name.lower().replace("-", "_").replace(".json", "").replace(".js", "").replace(" ", "_")

def get_field_name(row):
    return next((row.get(f"col_{i}") for i in range(7, 2, -1) if pd.notna(row.get(f"col_{i}"))), None)


def extract_path(row):
    path = []
    for col in [f"col_{i}" for i in range(1, 8)]:
        val = row.get(col)
        if pd.notna(val):
            path.append(str(val).strip())
    return path

def is_list_index(val):
    return str(val).isdigit()

# -----------------------
# Load and Clean Schema
# -----------------------
schema_path = '/mnt/c/Users/arodilla/Downloads/Structure output test - test.csv.csv'
schema_df = pd.read_csv(schema_path)
schema_df.columns = schema_df.columns.str.strip()

# -----------------------
# Generate Extractor Function
# -----------------------
def generate_df_function(file_folder_name: str, group: pd.DataFrame) -> str:
    original_root_key = file_folder_name
    func_name = f"{snake_case(file_folder_name)}_df"

    lines = [
        f"def {func_name}(file_input: List[str]) -> pd.DataFrame:",
        "    try:",
        "        with open(file_input[0], 'r', encoding='utf-8') as f:",
        "            data = json.load(f)",
        f"        root_data = get_in(data, '{original_root_key}')",
        "        if not root_data:",
        f"            print(f'⚠️ No data found at path: {original_root_key}')",
        "            return pd.DataFrame()",
        "",
        "        base_row = {}",
    ]

    static_fields = []
    list_blocks = {}  # key = tuple(list_path), value = set(fields)
    seen_static = set()

    for _, row in group.iterrows():
        path = extract_path(row)
        if path[0] == original_root_key:
            path = path[1:]

        for i, key in enumerate(path):
            if is_list_index(key):
                list_path = tuple(path[:i])  # use tuple for dict key
                field = path[i + 1] if i + 1 < len(path) else None
                if field:
                    list_blocks.setdefault(list_path, set()).add(field)
                break
        else:
            field = path[-1]
            if field not in seen_static:
                static_fields.append(path)
                seen_static.add(field)

    # Static field extraction
    for path in static_fields:
        field = path[-1]
        path_str = "', '".join(path)
        lines.append(f"        base_row['{field}'] = get_in(root_data, '{path_str}')")

    # If no list blocks, return single row
    if not list_blocks:
        lines.append("        return pd.DataFrame([base_row])")
    else:
        lines.append("        all_records = []")

        for list_path, fields in list_blocks.items():
            path_str = "', '".join(list_path)
            lines += [
                f"        items = get_list(root_data, '{path_str}')",
                "        for item in items:",
                "            row = base_row.copy()"
            ]
            for field in sorted(fields):
                lines.append(f"            row['{field}'] = item.get('{field}', '.*?')")
            lines.append("            all_records.append(row)")

        lines.append("        return pd.DataFrame(all_records)")

    lines += [
        "    except Exception as e:",
        f"        print(f'❌ Error in {func_name}:', e)",
        "        return pd.DataFrame()",
        ""
    ]

    return '\n'.join(lines)

# -----------------------
# Generate Donation Flow Function
# -----------------------
def generate_donation_flow(schema_df: pd.DataFrame) -> str:
    lines = [
        "def create_donation_flow(file_input: List[str]):",
        '    """Create donation flow from TikTok JSON."""',
        "    tables = []",
        ""
    ]
    for file_name, group in schema_df.groupby("col_1"):
        root_key = snake_case(file_name)
        func_name = f"{root_key}_df"
        lines += [
            f"    try:",
            f"        df = {func_name}(file_input)",
            f"        if not df.empty:",
            f"            tables.append(",
            f"                donation_table(name='{root_key}', df=df, title={{'en': '{root_key}'}})",
            f"            )",
            f"    except Exception as e:",
            f"        print(f'Error in {func_name}:', e)",
            ""
        ]
    lines += [
        "    if tables:",
        "        return donation_flow(id='tiktok', tables=tables)",
        "    else:",
        "        return None"
    ]
    return "\n".join(lines)

# -----------------------
# Generate & Write Output
# -----------------------
generated_functions = [
    generate_df_function(file_name, group)
    for file_name, group in schema_df.groupby("col_1")
]

donation_flow_code = generate_donation_flow(schema_df)

output_path = Path("/home/larodilla/BSC/WHATIF/what-if-data-donation/src/framework/processing/py/port/helpers/tiktok_generated_extractors.py")
with open(output_path, "w", encoding="utf-8") as f:
    f.write("# Auto-generated TikTok extractors\n\n")
    f.write("import pandas as pd\n")
    f.write("import json\n")
    f.write("import logging\n")
    f.write("from port.helpers.donation_flow import donation_table, donation_flow\n")
    f.write("from typing import List\n\n")
    f.write(inspect.getsource(get_in))
    f.write("\n")
    f.write(inspect.getsource(get_list))
    f.write("\n")
    f.write(inspect.getsource(get_dict))
    f.write("\n\n")
    f.write("\n\n".join(generated_functions))
    f.write("\n\n")
    f.write(donation_flow_code)

print(f"✅ Extractors written to {output_path}")

✅ Extractors written to /home/larodilla/BSC/WHATIF/what-if-data-donation/src/framework/processing/py/port/helpers/tiktok_generated_extractors.py


# ==================================================================================
# 4. Python code to dynamically create the Twitter script 
# ==================================================================================

In [ ]:
import pandas as pd
import json
import re
import io
import zipfile
import logging
import inspect

logger = logging.getLogger(__name__)

# --- Load Schema ---
schema_df = pd.read_csv('/mnt/c/Users/arodilla/OneDrive - Universitat de Barcelona/BSC/WHAT-IF/SCHEMA_DATA/Merged_structures_X.csv')
schema_df = schema_df.dropna(subset=["key"])
schema_df.columns = schema_df.columns.str.strip()

# --- Helper to Read .js files ---
def read_js(file_input: list[str], target_files: list[str]) -> list:
    """Extracts JSON content from matching .js files inside the ZIP."""
    extracted_data = []

    for zip_path in file_input:
        with zipfile.ZipFile(zip_path, "r") as z:
            for target_file in target_files:
                js_files = [f for f in z.namelist() if target_file in f]
                if js_files:
                    with z.open(js_files[0]) as raw_file:
                        with io.TextIOWrapper(raw_file, encoding="utf8") as text_file:
                            lines = text_file.readlines()
                        lines[0] = re.sub(r"^.*? = ", "", lines[0])  # remove variable assignment
                        try:
                            data = json.loads("".join(lines))
                            extracted_data.extend(data)  # assuming a list inside
                        except json.JSONDecodeError as e:
                            logger.error(f"Error decoding {target_file} in {zip_path}: {e}")

    return extracted_data

# --- Generate Function for Each .js File ---
def generate_df_function(file_folder_name: str, group: pd.DataFrame) -> str:
    table_base = file_folder_name.replace('.js', '').replace('-', '_')  # First level key from the file name
    func_name = f"{table_base}_df"
    file_clean = file_folder_name

    lines = []
    lines.append(f"def {func_name}(file_input: list[str]) -> pd.DataFrame:")
    lines.append(f"    data = read_js(file_input, ['{file_clean}'])")
    lines.append("")
    lines.append("    records = []")
    lines.append("    for item in data:")
    lines.append("        record = {}")

    # For each key in the schema, the first level is the file name (table_base)
    for key in group["key"].dropna():
        # Remove any non-word characters for the key, ensure it starts with a letter or underscore
        safe_key = re.sub(r"\W|^(?=\d)", "_", str(key))
        
        # The key is always nested with the first level being the file name
        nested_key = key  # Since the key will always be inside the first-level key (file name)
        
        lines.append(f"        {table_base} = item.get('{table_base}', {{}})")  # Extract the base part of the key (first level)
        lines.append(f"        record['{safe_key}'] = {table_base}.get('{nested_key}', '.*?')")  # Access the nested key

    lines.append("        records.append(record)")
    lines.append("")
    lines.append("    df = pd.DataFrame(records)")
    lines.append("    return df\n")

    return "\n".join(lines)

# --- Generate Donation Flow ---
def generate_donation_flow(schema_df: pd.DataFrame) -> str:
    lines = []
    lines.append("def create_donation_flow(file_input: list[str]):")
    lines.append('    """Create a donation flow for Twitter data."""')
    lines.append("    tables = []\n")

    for file_folder_name, group in schema_df.groupby("file_folder_name"):
        table_base = file_folder_name.replace('.js', '').replace('-', '_')
        func_name = f"{table_base}_df"

        lines.append(f"    try:")
        lines.append(f"        df = {func_name}(file_input)")
        lines.append(f"        if not df.empty:")
        lines.append(f"            tables.append(")
        lines.append(f"                donation_table(name='{table_base}', df=df, title={{'en': '{table_base}'}})")
        lines.append("            )")
        lines.append("    except Exception as e:")
        lines.append(f"        logger.error(f'Skipping {func_name}: {{e}}')\n")

    lines.append("    if tables:")
    lines.append("        return donation_flow(id='twitter', tables=tables)")
    lines.append("    else:")
    lines.append("        return None")

    return "\n".join(lines)

# --- Actually Generate Code ---

# Generate extractor functions
generated_functions = [
    generate_df_function(file_folder_name, group)
    for file_folder_name, group in schema_df.groupby("file_folder_name")
]

# Generate donation flow function
donation_flow_function_str = generate_donation_flow(schema_df)

# Save everything into a file
with open("/home/larodilla/BSC/WHATIF/what-if-data-donation/src/framework/processing/py/port/helpers/twitter_generated_extractors.py", "w", encoding="utf-8") as f:
    f.write("# Auto-generated Twitter extractors\n\n")
    f.write("import pandas as pd\n")
    f.write("import json\n")
    f.write("import logging\n")
    f.write("import io\n")
    f.write("import zipfile\n")
    f.write("import re\n")
    f.write("from port.helpers.donation_flow import donation_table, donation_flow\n\n")
    f.write("logger = logging.getLogger(__name__)\n\n")
    f.write(inspect.getsource(read_js))
    f.write("\n\n")
    f.write("\n\n".join(generated_functions))
    f.write("\n\n")
    f.write(donation_flow_function_str)